In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import datetime as dt
import xarray as xr
from math import atan2, log
import uuid
#read in email
text_file = open('C:/Users/gentemann/Google Drive/f_drive/docs/projects/saildrone/baja/docs/email_for_file.txt', "r")
email_str = text_file.readlines()
DOUBLE_FILLVALUE = np.finfo(np.float64).min


In [13]:
dir_in='f:/data/cruise_data/saildrone/noaa_cokelet/'
dir_out='f:/data/cruise_data/saildrone/noaa_cokelet/'
ISDP = 'Saildrone'
SST_type = 'SSTdepth'
Annex_version = '01.1'
File_version = '01.0'
droplist=['rowSize','temp_air_mean','temp_air_stddev','sal_mean','sal_stddev','temp_o2_mean','temp_o2_stddev']
dir_in='f:/data/cruise_data/saildrone/noaa_cokelet/'
file_list = ['saildrone_2015_arctic_126.nc','saildrone_2015_arctic_128.nc',
             'saildrone_2016_arctic_126.nc','saildrone_2016_arctic_128.nc']
platform_list = ['SD126','SD128','SD126','SD128']
for i in range(0,len(file_list)):
    filename_usv=dir_in+file_list[i]
    astr_platform=platform_list[i]
    astr_uuid = str(uuid.uuid4()) #'0f410de6-4ba5-4f79-af20-8a57a445f454'
    #read data back in, into two arrays one with time encoding and one without
    #adir='F:/data/cruise_data/saildrone/baja-2018/'
    #adir='C:/Users/gentemann/Desktop/data/'
    #fname=adir+'saildrone-gen_4-baja_2018-EP-sd1002-ALL-1_min-v1_with_loggers.nc'
    with xr.open_dataset(filename_usv,drop_variables=droplist) as ds:
        lons_usv=ds.longitude
        lats_usv=ds.latitude
        time_usv=ds.time
    with xr.open_dataset(filename_usv,drop_variables=droplist,decode_times=False) as ds_raw:
        time_usv_raw=ds_raw.time
    mint=time_usv.min().data
    maxt=time_usv.max().data
    ds.attrs['time_coverage_start']=str(np.datetime64(mint,'ms'))+'Z'
    ds.attrs['time_coverage_end']=str(np.datetime64(maxt,'ms'))+'Z'
    ilen=(len(ds.latitude['obs']))
    #dates_usv64=dataset_decodetime.TIME[0,:].values
    #time_usv_pd=pd.to_datetime(time_usv, unit='ns')
    print(ilen)
    R = 6373.0 #km
    lat1 = np.deg2rad(lats_usv[1:ilen])
    lon1 = np.deg2rad(lons_usv[1:ilen])
    lat2 = np.deg2rad(lats_usv[0:ilen-1])
    lon2 = np.deg2rad(lons_usv[0:ilen-1])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(a**.5, (1 - a)**.5)
    distance = R * c
    #add global attributes that are missing
    #some of these will need to be changed for new cruises
    dataset=ds
    gattrs = dataset.attrs.copy()
    gattrs['title'] = ds.title #astr_title
    gattrs['summary'] = 'none'
    gattrs['references'] = 'none'
    gattrs['institution'] = 'NOAA PMEL'
    gattrs['history'] = 'NOAA PMEL netcdf file was used to create this file'
    gattrs['comment'] = 'none'
    gattrs['license'] = 'CC-BY' #'free and open'
    gattrs['id'] = 'SSTdepth'
    gattrs['naming_authority'] = 'org.shipborne-radiometer'
    gattrs['product_version'] = '1.0'
    gattrs['uuid'] = astr_uuid 
    gattrs['l3r_version_id'] = '1.1' 
    gattrs['netcdf_version_id'] = '4.6.1'
    gattrs['date_created'] = dt.datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ") #yyyy-mm-ddThh:mm:ssZ
    gattrs['file_quality_level'] = 3
    gattrs['spatial_resolution'] = str(distance.mean().data*1000)+' m'
    gattrs['start_time'] = ds.time_coverage_start
    gattrs['stop_time'] = ds.time_coverage_end
    gattrs['northernmost_latitude'] = lats_usv.max().data
    gattrs['geospatial_lat_max'] = lats_usv.max().data
    gattrs['southernmost_latitude'] = lats_usv.min().data
    gattrs['geospatial_lat_min'] = lats_usv.min().data
    gattrs['easternmost_longitude'] = lons_usv.max().data
    gattrs['geospatial_lon_max'] = lons_usv.max().data
    gattrs['westernmost_longitude'] = lons_usv.min().data
    gattrs['geospatial_lon_min'] = lons_usv.min().data
    gattrs['geospatial_lat_units'] = 'degrees_north'
    gattrs['geospatial_lon_units'] = 'degrees_east'
    gattrs['source'] = 'SSTdepth, wind_speed'
    gattrs['platform'] = astr_platform
    #gattrs['sensor'] = str(dataset.temp_ctd_mean[0].device_name + ', ' + \
    #            dataset.UWND_MEAN[0].device_name)
    #Teledyne CTD (2632)_Teledyne CTD (2632)_2632, Gill Anemometer (172107)_Gill Anemometer (172107)_172107
    #print(gattrs['sensor'])
    gattrs['metadata_link'] = 'TBD'
    gattrs['keywords'] = 'Oceans > Ocean Temperature > Sea Surface Temperature'
    gattrs['keywords_vocabulary'] = 'NASA Global Change Master Directory (GCMD) Science Keywords'
    gattrs['acknowledgment'] = 'NOAA PMEL'
    gattrs['project'] = 'International Shipborne Radiometer Network'
    gattrs['publisher_name'] = 'The ISRN Project Office'
    gattrs['publisher_url'] = 'http://www.shipborne.radiometer.org'
    gattrs['publisher_email'] = ds.creator_email
    #print(gattrs['publisher_email'])
    gattrs['processing_level'] = '1.0'
    #del dataset.attrs['nodc_template_version']
    # convert names
    COORD_ATTR = 'lat lon time'
    #COORD_ATTR = 'time'
    FLOAT_FILLVALUE = np.finfo(np.float32).min
    VAR_TRANSLATE = {
        'temp_ctd_mean': 'sea_water_temperature', #   'COG': 'course_over_ground',
        'latitude': 'lat',
        'longitude': 'lon',
        'temp_ctd_stddev':'sst_total_uncertainty',
        'wind_speed_mean':'wind_speed',
        'wind_direction_mean':'wind_direction',
    #    'wind_speed_stddev':'wind_speed_total_uncertainty'
      #  'HDG': 'true_bearing',
      #  'ROLL': 'platform_roll',
      #  'PITCH': 'platform_pitch',
      #  'SOG': 'speed_over_ground',
    }
    # copy variables from source dataset
    varrays = {}
    for v in dataset.data_vars:
        if v in VAR_TRANSLATE:
           # print(v)
            # set attributes
            vattrs = dataset[v].attrs.copy()
            if v not in ['latitude', 'longitude', 'time']:
                vattrs["coordinates"] = COORD_ATTR
            vattrs["_FillValue"] = FLOAT_FILLVALUE

            values = np.ma.fix_invalid(dataset[v].values)
            if v == 'temp_ctd_mean':
                # convert SST to Kelvin
                values += 273.15        
       #     if v == 'wind_speed_mean':
       #         # convert SST to Kelvin
       #         values = values*0.514444  #from knots to m/s  
       #         WS_height=5
       #         values = (values*log(10./1e-4))/log(WS_height/1e-4)

            # create new data array
            varrays[VAR_TRANSLATE[v]] = xr.DataArray(
                values.filled(FLOAT_FILLVALUE).astype(np.float32,copy=False),
                dims=['time'],
                attrs=vattrs
            )
    for v in dataset.coords:
        if v in VAR_TRANSLATE:
           # print(v)
            # set attributes
            vattrs = dataset[v].attrs.copy()
            if v not in ['latitude', 'longitude', 'time']:
                vattrs["coordinates"] = COORD_ATTR
            vattrs["_FillValue"] = FLOAT_FILLVALUE

            values = np.ma.fix_invalid(dataset[v].values)
            # create new data array
            varrays[VAR_TRANSLATE[v]] = xr.DataArray(
                values.filled(FLOAT_FILLVALUE).astype(np.float32,copy=False),
                dims=['time'],
                attrs=vattrs
            )


    # 1. wind speed
    vattrs = dataset.wind_speed_mean.attrs.copy()
    vattrs['standard_name'] = 'wind_speed'
    vattrs['long_name'] = 'wind_speed'
    vattrs['valid_min'] = 0
    vattrs['valid_max'] = 100
    vattrs["_FillValue"] = FLOAT_FILLVALUE
    vattrs['source'] = 'anemometer'
    vattrs['comment'] = 'Instrument located at to of Saildrone mast at 5 m' + '.  This was adjusted ' +\
                    'to 10 m as ws_10m = ws*log(10./1e-4))/log(WS_height/1e-4'
    vattrs['height'] = '10 m' #str(str(dataset.UWND_MEAN.installed_height)+' m')
    vattrs["coordinates"] = COORD_ATTR
    WS=(dataset['wind_speed_mean'].values)*.51444  #from knots to m/s
    WS_height=int(5)
    WS_10m = (WS*log(10./1e-4))/log(WS_height/1e-4)
    varrays['wind_speed'] = xr.DataArray(
        WS_10m.astype(np.float32,copy=False),
        dims=['time'],
        attrs=vattrs
        )

    # 2. wind direction
    vattrs = dataset.wind_direction_mean.attrs.copy()
    vattrs['standard_name'] = 'wind_to_direction'
    vattrs['long_name'] = 'local wind direction'
    vattrs['valid_min'] = -180
    vattrs['valid_max'] = 180
    vattrs["_FillValue"] = FLOAT_FILLVALUE
    vattrs['units'] = 'degrees'
    vattrs['source'] = 'anemometer'
    vattrs['height'] = '5 m'
    vattrs["coordinates"] = COORD_ATTR
    WD=dataset['wind_direction_mean'].values
    varrays['wind_direction'] = xr.DataArray(
        WD.astype(np.float32,copy=False),
        dims=['time'],
        attrs=vattrs
        )

    # 3. time
    vattrs = ds_raw.time.attrs.copy()
    vattrs["_FillValue"] = DOUBLE_VALUE
    values = np.ma.fix_invalid(ds_raw.time.values)
    vtime = xr.DataArray(
        values.filled(FLOAT_FILLVALUE).astype(np.float64,copy=False),
        dims=['time'],
        attrs=vattrs
        )


    # 4. Quality level
    flag_bytes=np.byte((0,1,2,3,4,5)) #bytearray([0,1,2,3,4,5])
    iqual_byte = np.ones(shape=time_usv.shape, dtype='b')*5  #change byte to b1
    #iqual_byte[:itow_mask1] = 2  #set at top of file from looking at data
    #iqual_byte[itow_mask2:] = 2
    vattrs = {
        'long_name': 'measurement quality value',
        'flag_meanings': 'no_data bad_data worst_quality low_quality acceptable_quality best_quality',
        'flag_values': flag_bytes,
        '_FillValue': -128
    }
    varrays['quality_level'] = xr.DataArray(
        iqual_byte,
        dims=['time'],
        attrs=vattrs
        )


    # create Dataset and save
    l3r = xr.Dataset(
        varrays,
        coords = {
    #        'time': time_usv
            'time': vtime
        },
        attrs=gattrs
    )

    # embellishments
    l3r.sea_water_temperature.attrs['valid_min']=260.0
    l3r.sea_water_temperature.attrs['valid_max']=320.0
    l3r.sea_water_temperature.attrs['units']='kelvin'
    l3r.sea_water_temperature.attrs['long_name']='sea surface depth temperature at 0.6m'
    l3r.time.attrs['standard_name']='time'
    l3r.time.attrs['long_name']='time'
    l3r.lon.attrs['standard_name']='longitude'
    l3r.lon.attrs['long_name']='longitude'
    l3r.lat.attrs['long_name']='latitude'
    l3r.lat.attrs['standard_name']='latitude'
    #l3r.true_bearing.attrs['long_name']='platform true bearing'
    #l3r.true_bearing.attrs['standard_name']='platform_orientation'
    #l3r.speed_over_ground.attrs['long_name']='platform speed over ground'
    l3r.sst_total_uncertainty.attrs['standard_name']='sea_water_temperature standard error'
    l3r.sst_total_uncertainty.attrs['long_name']=' sea water temperature total uncertainty'
    l3r.sst_total_uncertainty.attrs['valid_min']=0.0
    l3r.sst_total_uncertainty.attrs['valid_max']=2.0
    l3r.sst_total_uncertainty.attrs['units']='kelvin'
    l3r.sea_water_temperature.attrs['colorBarMaximum']=320.0
    l3r.sea_water_temperature.attrs['colorBarMinimum']=260.0

    indicative_date_time=pd.to_datetime(str(time_usv[0].data)).strftime("%Y%m%d%H%M%S")
    #Product_string = str(ds.TEMP_CTD_MEAN.vendor_name) + '_' + str(ds.TEMP_CTD_MEAN.serial_number)
#    Product_string = 'Teledyne'
    Product_string = astr_platform #str(ds.TEMP_CTD_MEAN.model_number) + '_' + str(ds.TEMP_CTD_MEAN.serial_number)

    filename_L3R = dir_in + indicative_date_time + \
        '-' + ISDP + '-' + 'L3R' + '-' + SST_type + '-' +Product_string+ '-v' +Annex_version+ '-fv' +File_version+ '.nc'
    filename_L3R

    #l3r.to_netcdf(filename_L3R)

    #for some reason the time not decoded is off by about 28 seconds so reset to original decoded time here
    #figured out it was the converstion to int32 rather than 64
    #l3r['time']=ds.time.data                 
    l3r.to_netcdf(filename_L3R)
    del l3r,vattrs,gattrs,dataset


114374
132544
149399
149348


In [14]:
dir_in='f:/data/cruise_data/saildrone/noaa_cokelet/'
dir_out='f:/data/cruise_data/saildrone/noaa_cokelet/'
ISDP = 'Saildrone'
SST_type = 'SSTdepth'
Annex_version = '01.1'
File_version = '01.0'
droplist=['rowSize','temp_air_mean','temp_air_stddev','sal_mean','sal_stddev','temp_o2_mean','temp_o2_stddev']
dir_in='f:/data/cruise_data/saildrone/noaa_cokelet/'
file_list = ['saildrone_data_2017_arctic_1001_02.nc', 'saildrone_data_2017_arctic_1002_02.nc', 'saildrone_data_2017_arctic_1003_02.nc']
platform_list = ['SD1001','SD1002','SD1003']
for i in range(0,len(file_list)):
    filename_usv=dir_in+file_list[i]
    astr_platform=platform_list[i]
    astr_uuid = str(uuid.uuid4()) #'0f410de6-4ba5-4f79-af20-8a57a445f454'
    #read data back in, into two arrays one with time encoding and one without
    #adir='F:/data/cruise_data/saildrone/baja-2018/'
    #adir='C:/Users/gentemann/Desktop/data/'
    #fname=adir+'saildrone-gen_4-baja_2018-EP-sd1002-ALL-1_min-v1_with_loggers.nc'
    with xr.open_dataset(filename_usv,drop_variables=droplist) as ds:
        lons_usv=ds.longitude
        lats_usv=ds.latitude
        time_usv=ds.time
    with xr.open_dataset(filename_usv,drop_variables=droplist,decode_times=False) as ds_raw:
        time_usv_raw=ds_raw.time
    mint=time_usv.min().data
    maxt=time_usv.max().data
    ds.attrs['time_coverage_start']=str(np.datetime64(mint,'ms'))+'Z'
    ds.attrs['time_coverage_end']=str(np.datetime64(maxt,'ms'))+'Z'
    ilen=(len(ds.latitude['obs']))
    #dates_usv64=dataset_decodetime.TIME[0,:].values
    #time_usv_pd=pd.to_datetime(time_usv, unit='ns')
    print(ilen)
    R = 6373.0 #km
    lat1 = np.deg2rad(lats_usv[1:ilen])
    lon1 = np.deg2rad(lons_usv[1:ilen])
    lat2 = np.deg2rad(lats_usv[0:ilen-1])
    lon2 = np.deg2rad(lons_usv[0:ilen-1])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(a**.5, (1 - a)**.5)
    distance = R * c
    #add global attributes that are missing
    #some of these will need to be changed for new cruises
    dataset=ds
    gattrs = dataset.attrs.copy()
    gattrs['title'] = ds.title #astr_title
    gattrs['summary'] = 'none'
    gattrs['references'] = 'none'
    gattrs['institution'] = 'NOAA PMEL'
    gattrs['history'] = 'NOAA PMEL netcdf file was used to create this file'
    gattrs['comment'] = 'none'
    gattrs['license'] = 'CC-BY' #'free and open'
    gattrs['id'] = 'SSTdepth'
    gattrs['naming_authority'] = 'org.shipborne-radiometer'
    gattrs['product_version'] = '1.0'
    gattrs['uuid'] = astr_uuid 
    gattrs['l3r_version_id'] = '1.1' 
    gattrs['netcdf_version_id'] = '4.6.1'
    gattrs['date_created'] = dt.datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ") #yyyy-mm-ddThh:mm:ssZ
    gattrs['file_quality_level'] = 3
    gattrs['spatial_resolution'] = str(distance.mean().data*1000)+' m'
    gattrs['start_time'] = ds.time_coverage_start
    gattrs['stop_time'] = ds.time_coverage_end
    gattrs['northernmost_latitude'] = lats_usv.max().data
    gattrs['geospatial_lat_max'] = lats_usv.max().data
    gattrs['southernmost_latitude'] = lats_usv.min().data
    gattrs['geospatial_lat_min'] = lats_usv.min().data
    gattrs['easternmost_longitude'] = lons_usv.max().data
    gattrs['geospatial_lon_max'] = lons_usv.max().data
    gattrs['westernmost_longitude'] = lons_usv.min().data
    gattrs['geospatial_lon_min'] = lons_usv.min().data
    gattrs['geospatial_lat_units'] = 'degrees_north'
    gattrs['geospatial_lon_units'] = 'degrees_east'
    gattrs['source'] = 'SSTdepth, wind_speed'
    gattrs['platform'] = astr_platform
    gattrs['sensor'] = str(dataset.TEMP_CTD_MEAN[0].model_number + ', ' + \
                dataset.VWND_MEAN[0].model_number)
    #Teledyne CTD (2632)_Teledyne CTD (2632)_2632, Gill Anemometer (172107)_Gill Anemometer (172107)_172107
    #print(gattrs['sensor'])
    gattrs['metadata_link'] = 'TBD'
    gattrs['keywords'] = 'Oceans > Ocean Temperature > Sea Surface Temperature'
    gattrs['keywords_vocabulary'] = 'NASA Global Change Master Directory (GCMD) Science Keywords'
    gattrs['acknowledgment'] = 'NOAA PMEL'
    gattrs['project'] = 'International Shipborne Radiometer Network'
    gattrs['publisher_name'] = 'The ISRN Project Office'
    gattrs['publisher_url'] = 'http://www.shipborne.radiometer.org'
    gattrs['publisher_email'] = ds.creator_email
    #print(gattrs['publisher_email'])
    gattrs['processing_level'] = '1.0'
    #del dataset.attrs['nodc_template_version']
    # convert names
    COORD_ATTR = 'lat lon time'
    #COORD_ATTR = 'time'
    FLOAT_FILLVALUE = np.finfo(np.float32).min
    VAR_TRANSLATE = {
        'TEMP_CTD_MEAN': 'sea_water_temperature', #   'COG': 'course_over_ground',
        'latitude': 'lat',
        'longitude': 'lon',
        'TEMP_CTD_STDDEV':'sst_total_uncertainty',
  #      'wind_speed_mean':'wind_speed',
  #      'wind_direction_mean':'wind_direction',
    #    'wind_speed_stddev':'wind_speed_total_uncertainty'
      #  'HDG': 'true_bearing',
      #  'ROLL': 'platform_roll',
      #  'PITCH': 'platform_pitch',
      #  'SOG': 'speed_over_ground',
    }
    # copy variables from source dataset
    varrays = {}
    for v in dataset.data_vars:
        if v in VAR_TRANSLATE:
           # print(v)
            # set attributes
            vattrs = dataset[v].attrs.copy()
            if v not in ['latitude', 'longitude', 'time']:
                vattrs["coordinates"] = COORD_ATTR
            vattrs["_FillValue"] = FLOAT_FILLVALUE

            values = np.ma.fix_invalid(dataset[v].values)
            if v == 'TEMP_CTD_MEAN':
                # convert SST to Kelvin
                values += 273.15        
       #     if v == 'wind_speed_mean':
       #         # convert SST to Kelvin
       #         values = values*0.514444  #from knots to m/s  
       #         WS_height=5
       #         values = (values*log(10./1e-4))/log(WS_height/1e-4)

            # create new data array
            varrays[VAR_TRANSLATE[v]] = xr.DataArray(
                values.filled(FLOAT_FILLVALUE).astype(np.float32,copy=False),
                dims=['time'],
                attrs=vattrs
            )
    for v in dataset.coords:
        if v in VAR_TRANSLATE:
           # print(v)
            # set attributes
            vattrs = dataset[v].attrs.copy()
            if v not in ['latitude', 'longitude', 'time']:
                vattrs["coordinates"] = COORD_ATTR
            vattrs["_FillValue"] = FLOAT_FILLVALUE

            values = np.ma.fix_invalid(dataset[v].values)
            # create new data array
            varrays[VAR_TRANSLATE[v]] = xr.DataArray(
                values.filled(FLOAT_FILLVALUE).astype(np.float32,copy=False),
                dims=['time'],
                attrs=vattrs
            )


    # 1. wind speed
    vattrs = dataset.UWND_MEAN.attrs.copy()
#    vattrs = dataset.wind_speed_mean.attrs.copy()
    vattrs['standard_name'] = 'wind_speed'
    vattrs['long_name'] = 'wind_speed'
    vattrs['valid_min'] = 0
    vattrs['valid_max'] = 100
    vattrs["_FillValue"] = FLOAT_FILLVALUE
    vattrs['source'] = 'anemometer'
    vattrs['comment'] = 'Instrument located at to of Saildrone mast at 5 m' + '.  This was adjusted ' +\
                    'to 10 m as ws_10m = ws*log(10./1e-4))/log(WS_height/1e-4'
    vattrs['height'] = '10 m' #str(str(dataset.UWND_MEAN.installed_height)+' m')
    vattrs["coordinates"] = COORD_ATTR
    #WS=(dataset['wind_speed_mean'].values)*.51444  #from knots to m/s
    WS=(dataset['UWND_MEAN'].values**2 + dataset['VWND_MEAN'].values**2)**.5
    WS_height=int(5)
    WS_10m = (WS*log(10./1e-4))/log(WS_height/1e-4)
    varrays['wind_speed'] = xr.DataArray(
        WS_10m.astype(np.float32,copy=False),
        dims=['time'],
        attrs=vattrs
        )

    # 2. wind direction
    vattrs = dataset.UWND_MEAN.attrs.copy()
    vattrs['standard_name'] = 'wind_to_direction'
    vattrs['long_name'] = 'local wind direction'
    vattrs['valid_min'] = -180
    vattrs['valid_max'] = 180
    vattrs["_FillValue"] = FLOAT_FILLVALUE
    vattrs['units'] = 'degrees'
    vattrs['source'] = 'anemometer'
    vattrs['height'] = '5 m'
    vattrs["coordinates"] = COORD_ATTR
    WD=np.arctan2(dataset.VWND_MEAN.values, dataset.UWND_MEAN.values)*180/3.14159
    WD=np.where(WD<0,WD+360,WD)
#    WD=dataset['wind_direction_mean'].values
    varrays['wind_direction'] = xr.DataArray(
        WD.astype(np.float32,copy=False),
        dims=['time'],
        attrs=vattrs
        )

    # 3. time
    vattrs = ds_raw.time.attrs.copy()
    vattrs["_FillValue"] = DOUBLE_FILLVALUE
    values = np.ma.fix_invalid(ds_raw.time.values)
    vtime = xr.DataArray(
        values.filled(FLOAT_FILLVALUE).astype(np.float64,copy=False),
        dims=['time'],
        attrs=vattrs
        )


    # 4. Quality level
    flag_bytes=np.byte((0,1,2,3,4,5)) #bytearray([0,1,2,3,4,5])
    iqual_byte = np.ones(shape=time_usv.shape, dtype='b')*5  #change byte to b1
    #iqual_byte[:itow_mask1] = 2  #set at top of file from looking at data
    #iqual_byte[itow_mask2:] = 2
    vattrs = {
        'long_name': 'measurement quality value',
        'flag_meanings': 'no_data bad_data worst_quality low_quality acceptable_quality best_quality',
        'flag_values': flag_bytes,
        '_FillValue': -128
    }
    varrays['quality_level'] = xr.DataArray(
        iqual_byte,
        dims=['time'],
        attrs=vattrs
        )


    # create Dataset and save
    l3r = xr.Dataset(
        varrays,
        coords = {
    #        'time': time_usv
            'time': vtime
        },
        attrs=gattrs
    )

    # embellishments
    l3r.sea_water_temperature.attrs['valid_min']=260.0
    l3r.sea_water_temperature.attrs['valid_max']=320.0
    l3r.sea_water_temperature.attrs['units']='kelvin'
    l3r.sea_water_temperature.attrs['long_name']='sea surface depth temperature at 0.6m'
    l3r.time.attrs['standard_name']='time'
    l3r.time.attrs['long_name']='time'
    l3r.lon.attrs['standard_name']='longitude'
    l3r.lon.attrs['long_name']='longitude'
    l3r.lat.attrs['long_name']='latitude'
    l3r.lat.attrs['standard_name']='latitude'
    #l3r.true_bearing.attrs['long_name']='platform true bearing'
    #l3r.true_bearing.attrs['standard_name']='platform_orientation'
    #l3r.speed_over_ground.attrs['long_name']='platform speed over ground'
    l3r.sst_total_uncertainty.attrs['standard_name']='sea_water_temperature standard error'
    l3r.sst_total_uncertainty.attrs['long_name']=' sea water temperature total uncertainty'
    l3r.sst_total_uncertainty.attrs['valid_min']=0.0
    l3r.sst_total_uncertainty.attrs['valid_max']=2.0
    l3r.sst_total_uncertainty.attrs['units']='kelvin'
    l3r.sea_water_temperature.attrs['colorBarMaximum']=320.0
    l3r.sea_water_temperature.attrs['colorBarMinimum']=260.0

    indicative_date_time=pd.to_datetime(str(time_usv[0].data)).strftime("%Y%m%d%H%M%S")
    Product_string = astr_platform #str(ds.TEMP_CTD_MEAN.model_number) + '_' + str(ds.TEMP_CTD_MEAN.serial_number)
    #Product_string = 'Teledyne'
    filename_L3R = dir_in + indicative_date_time + \
        '-' + ISDP + '-' + 'L3R' + '-' + SST_type + '-' +Product_string+ '-v' +Annex_version+ '-fv' +File_version+ '.nc'
    
    
    filename_L3R

    #l3r.to_netcdf(filename_L3R)

    #for some reason the time not decoded is off by about 28 seconds so reset to original decoded time here
    #l3r['time']=ds.time.data                 
    l3r.to_netcdf(filename_L3R)
    print('out:',filename_L3R)
    del l3r,vattrs,gattrs,dataset


106799
out: f:/data/cruise_data/saildrone/noaa_cokelet/20170717000000-Saildrone-L3R-SSTdepth-SD1001-v01.1-fv01.0.nc
106737


C:\Users\gentemann\Anaconda3\lib\site-packages\ipykernel_launcher.py:187: RuntimeWarning: invalid value encountered in less
C:\Users\gentemann\Anaconda3\lib\site-packages\ipykernel_launcher.py:187: RuntimeWarning: invalid value encountered in less


out: f:/data/cruise_data/saildrone/noaa_cokelet/20170717000000-Saildrone-L3R-SSTdepth-SD1002-v01.1-fv01.0.nc
106798
out: f:/data/cruise_data/saildrone/noaa_cokelet/20170717000000-Saildrone-L3R-SSTdepth-SD1003-v01.1-fv01.0.nc


C:\Users\gentemann\Anaconda3\lib\site-packages\ipykernel_launcher.py:187: RuntimeWarning: invalid value encountered in less


In [5]:
#dataset

<xarray.Dataset>
Dimensions:          (obs: 106799, trajectory: 1)
Coordinates:
  * trajectory       (trajectory) object '1001.0'
    longitude        (obs) float64 -166.6 -166.6 -166.6 -166.6 -166.6 -166.6 ...
    latitude         (obs) float64 53.89 53.89 53.89 53.89 53.89 53.89 53.89 ...
    time             (obs) datetime64[ns] 2017-07-17 2017-07-17T00:01:00 ...
Dimensions without coordinates: obs
Data variables:
    TEMP_O2_STDDEV   (obs) float64 ...
    UWND_MEAN        (obs) float64 ...
    SAL_STDDEV       (obs) float64 ...
    TEMP_AIR_MEAN    (obs) float64 ...
    SAL_MEAN         (obs) float64 ...
    WWND_STDDEV      (obs) float64 ...
    WWND_MEAN        (obs) float64 ...
    TEMP_O2_MEAN     (obs) float64 ...
    TEMP_CTD_STDDEV  (obs) float64 ...
    TEMP_AIR_STDDEV  (obs) float64 ...
    VWND_STDDEV      (obs) float64 ...
    TEMP_CTD_MEAN    (obs) float64 ...
    VWND_MEAN        (obs) float64 ...
    UWND_STDDEV      (obs) float64 ...
    SOG              (obs) float6